In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
w = pd.read_table(PATH["w.tsv"], index_col=0)

w.columns.name = "Factor"

h = pd.read_table(PATH["h.tsv"], index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
for w_or_h, df in (("w", w.T), ("h", h)):

    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        df,
        SETTING["hcc_ks"],
        n_job=SETTING["max_n_job"],
        distance_function=ccal.compute_correlation_distance,
        n_clustering=SETTING["hcc_n_clustering"],
        plot_df=df.shape[1] < SETTING["plot_cluster_max_size"],
        directory_path=PATH["{}|hcc/".format(w_or_h)],
    )

In [ ]:
w_or_h = "w"

new_clusters = ()

In [ ]:
if len(new_clusters):

    cluster_x_column = pd.read_table(
        PATH["{}|cluster_x_column.tsv".format(w_or_h)], index_col=0
    )

    print(cluster_x_column.index)

    if (
        cluster_x_column.index.str.lstrip("C").astype(int).sort_values()
        == cluster_x_column.index.str.lstrip("C").astype(int)
    ).all():

        cluster_x_column.index = pd.Index(
            ("C{}".format(i) for i in new_clusters), name=cluster_x_column.index.name
        )

        cluster_x_column.to_csv(
            PATH["{}|cluster_x_column.tsv".format(w_or_h)], sep="\t"
        )

        print(cluster_x_column.index)

    else:

        print("Indices may be already updated.")